# Notebook 03: Natural Language Processing - Text Summarization

**Learning Objectives:**
- Understand automatic text summarization
- Use seq2seq models for summarization
- Generate concise summaries from long documents
- Compare extractive vs abstractive summarization

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **CPU (Small)** | sshleifer/distilbart-cnn-12-6 | 1.2GB | 4GB | 4GB RAM, CPU | Faster inference |
| **GPU (Medium)** | facebook/bart-large-cnn | 1.6GB | 6GB | 8GB VRAM (RTX 4080) | Better quality |

### Software Requirements
- Python 3.8+
- Libraries: `transformers`, `torch`
- See `requirements.txt` for full list

## Overview

**Text Summarization** condenses long documents into shorter versions while preserving key information.

**Types:**
- **Extractive**: Selects important sentences from the original text
- **Abstractive**: Generates new sentences that capture the meaning (what we'll use)

**Use Cases:**
- News article summarization
- Research paper abstracts
- Meeting notes
- Document digests

**How it works:**
1. Encoder reads and understands the full text
2. Decoder generates a concise summary
3. Model was trained on pairs of (document, summary)

## Expected Behaviors

### First Time Running
- **Model Download**: ~1.2GB for distilbart (~3-5 minutes download)
- Largest model so far in the NLP notebooks
- Cached for future runs in `~/.cache/huggingface/hub/`

### Setup Cell Output
```
PyTorch version: 2.x.x
CUDA available: True/False
```

### Model Loading Time
- **CPU**: 10-15 seconds to load model into memory
- **GPU**: 5-10 seconds
- Loading message appears: `Loading sshleifer/distilbart-cnn-12-6...`

### Summarization Output
```python
{
  'summary_text': 'Large language models have demonstrated impressive capabilities...'
}
```

### Summary Quality Expectations
- **Good summaries**: Capture main points from original text
- **Abstractive**: Uses new words/phrases, not just copying sentences
- **Coherent**: Reads naturally, not fragmented
- **Length**: Typically 20-30% of original text length

### Performance Benchmarks
- **Short text** (100 words):
  - CPU: 2-4 seconds
  - GPU: 0.5-1 second
- **Medium text** (300 words):
  - CPU: 5-8 seconds
  - GPU: 1-2 seconds
- **Long text** (500+ words):
  - CPU: 10-15 seconds
  - GPU: 2-4 seconds

### Beam Search Impact
- **num_beams=1**: Fastest, lower quality
- **num_beams=4**: Good balance (default)
- **num_beams=8**: Best quality, 2x slower

### Common Observations
- Summaries may occasionally omit minor details (expected)
- Works best with factual, structured text (news, articles)
- Creative/narrative text may have mixed results
- Very short inputs (<50 words) may produce minimal summaries

## Setup and Installation

In [ ]:
# Import required libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, set_seed
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(1103)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Model Selection

In [ ]:
# CHOOSE YOUR MODEL:

# Option 1: CPU-friendly (recommended for beginners)
MODEL_NAME = "sshleifer/distilbart-cnn-12-6"  # 1.2GB, distilled BART

# Option 2: GPU-optimized (uncomment if you have RTX 4080 or similar)
# MODEL_NAME = "facebook/bart-large-cnn"  # 1.6GB, better quality

print(f"Selected model: {MODEL_NAME}")

## Method 1: Using Pipeline (Simplest)

In [ ]:
# Create summarization pipeline
print(f"Loading {MODEL_NAME}...")
summarizer = pipeline(
    "summarization",
    model=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)

### Basic Summarization

In [ ]:
# Example article
article = """
The field of artificial intelligence has seen remarkable progress in recent years, 
particularly in the domain of natural language processing. Large language models, 
which are trained on vast amounts of text data, have demonstrated impressive capabilities 
in understanding and generating human-like text. These models use transformer architectures, 
which employ attention mechanisms to process sequential data more effectively than previous 
approaches. The transformer architecture was introduced in the landmark paper "Attention Is All 
You Need" in 2017. Since then, models like BERT, GPT, and their variants have revolutionized 
numerous NLP tasks including translation, summarization, question answering, and text generation. 
However, these models also raise important concerns about computational costs, environmental impact, 
and potential misuse. Researchers are now focusing on making these models more efficient, 
interpretable, and aligned with human values.
"""

# Generate summary
summary = summarizer(article, max_length=130, min_length=30, do_sample=False)

print("Original text length:", len(article.split()))
print("\n=== SUMMARY ===")
print(summary[0]['summary_text'])
print("\nSummary length:", len(summary[0]['summary_text'].split()))

### Controlling Summary Length

In [ ]:
# Generate summaries of different lengths
lengths = [(50, 20), (100, 40), (150, 60)]  # (max_length, min_length)

print("=== Summaries of Different Lengths ===")
for max_len, min_len in lengths:
    summary = summarizer(article, max_length=max_len, min_length=min_len, do_sample=False)
    print(f"\n[{min_len}-{max_len} tokens]:")
    print(summary[0]['summary_text'])

## Method 2: Using Model and Tokenizer Directly

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on: {device}")

In [ ]:
# Generate summary with more control
text = """
Climate change is one of the most pressing challenges facing humanity today. 
Rising global temperatures are causing ice caps to melt, sea levels to rise, 
and extreme weather events to become more frequent. Scientists warn that without 
immediate action to reduce greenhouse gas emissions, the consequences could be 
catastrophic. Renewable energy sources like solar and wind power offer promising 
solutions, but transitioning away from fossil fuels requires coordinated global 
effort and significant investment. Many countries have committed to achieving 
net-zero emissions by 2050, but current progress suggests more ambitious actions 
are needed.
"""

# Tokenize
inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate summary
summary_ids = model.generate(
    inputs.input_ids,
    max_length=100,
    min_length=30,
    length_penalty=2.0,  # Encourages longer summaries
    num_beams=4,         # Beam search for better quality
    early_stopping=True
)

# Decode
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("=== Climate Change Summary ===")
print(summary)

## Practical Applications

### Example 1: News Article Summarization

In [ ]:
news_article = """
Tech giant announces breakthrough in quantum computing. The company revealed a new 
quantum processor that achieved quantum supremacy, solving complex problems that would 
take classical computers thousands of years to complete. The 70-qubit processor 
demonstrated error rates low enough for practical applications. Industry experts believe 
this development could revolutionize fields like drug discovery, financial modeling, and 
cryptography. However, practical quantum computers for everyday use are still years away. 
The company plans to make the technology available to researchers through cloud access 
next year. Competitors are racing to develop their own quantum systems, with several 
other tech companies announcing similar initiatives. The quantum computing market is 
expected to grow to $65 billion by 2030 according to industry analysts.
"""

summary = summarizer(news_article, max_length=60, min_length=25, do_sample=False)

print("=== NEWS SUMMARY ===")
print(summary[0]['summary_text'])

### Example 2: Meeting Notes Summarization

In [ ]:
meeting_notes = """
The quarterly team meeting began with a review of the project timeline. Sarah reported 
that the mobile app development is on track for the March release. However, backend 
integration is facing delays due to API compatibility issues. Mike volunteered to work 
with the infrastructure team to resolve these problems by next week. The marketing team 
presented their campaign strategy, proposing a budget increase of 15% for social media 
advertising. After discussion, the team agreed to the proposal pending approval from 
upper management. Action items include: Sarah to complete user testing by Friday, 
Mike to fix API issues by next Tuesday, and Jennifer to submit the budget proposal 
by end of week. The next meeting is scheduled for two weeks from today.
"""

summary = summarizer(meeting_notes, max_length=80, min_length=30, do_sample=False)

print("=== MEETING SUMMARY ===")
print(summary[0]['summary_text'])

### Example 3: Batch Summarization

In [ ]:
# Summarize multiple documents
documents = [
    """
    A new study shows that regular exercise can significantly improve cognitive function 
    in older adults. Researchers followed 500 participants over three years, finding that 
    those who engaged in moderate aerobic exercise for 30 minutes daily showed 25% better 
    memory retention than the control group.
    """,
    """
    Electric vehicle sales reached a record high this quarter, with major manufacturers 
    reporting 40% year-over-year growth. The surge is attributed to improved battery 
    technology, expanded charging infrastructure, and government incentives. Industry 
    analysts predict EVs will account for 50% of new car sales by 2030.
    """,
    """
    A recent archaeological discovery in Egypt has uncovered a previously unknown pharaoh's 
    tomb. The tomb, dating back 3,400 years, contains remarkably well-preserved artifacts 
    and hieroglyphics that provide new insights into ancient Egyptian society. Experts 
    believe this finding could rewrite parts of Egyptian history.
    """
]

summaries = summarizer(documents, max_length=50, min_length=20, do_sample=False)

print("=== BATCH SUMMARIZATION ===")
for i, (doc, summary) in enumerate(zip(documents, summaries), 1):
    print(f"\nDocument {i}:")
    print(f"Summary: {summary['summary_text']}")

## Performance Benchmarking

## State-of-the-Art Open Models (Not Covered)

While we've used DistilBART and BART for this tutorial, there are several more powerful state-of-the-art summarization models available. These models offer better quality but require significantly more computational resources.

### Top SOTA Summarization Models

#### 1. 📄 PEGASUS (Google)
**Google's state-of-the-art abstractive summarization model**
- **Why it's special**: Pre-trained specifically for summarization using "gap sentence generation"
- **Performance**: Often achieves best ROUGE scores on benchmark datasets
- **Model Card**: [google/pegasus-xsum](https://huggingface.co/google/pegasus-xsum)
- **Paper**: [PEGASUS: Pre-training with Extracted Gap-sentences](https://arxiv.org/abs/1912.08777)
- **Size**: ~2.3GB (570M parameters)

#### 2. 🔧 T5-Large/XL (Google)
**Text-to-Text Transfer Transformer for summarization**
- **Why it's special**: Frames all NLP tasks as text-to-text, extremely versatile
- **Performance**: Excellent on diverse summarization tasks (news, scientific papers, dialogues)
- **Model Card**: [t5-large](https://huggingface.co/t5-large), [t5-3b](https://huggingface.co/t5-3b)
- **Paper**: [Exploring the Limits of Transfer Learning](https://arxiv.org/abs/1910.10683)
- **Size**: 3GB (T5-large), 11GB (T5-3B)

#### 3. 🎯 BART-Large (Facebook)
**Full-sized BART for higher quality summaries**
- **Why it's special**: Larger version of the model we used, better coherence and accuracy
- **Performance**: Strong baseline for abstractive summarization
- **Model Card**: [facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)
- **Paper**: [BART: Denoising Sequence-to-Sequence Pre-training](https://arxiv.org/abs/1910.13461)
- **Size**: 1.6GB (406M parameters)

#### 4. 📚 LED (Longformer Encoder-Decoder)
**Specialized for long document summarization**
- **Why it's special**: Can handle documents up to 16,384 tokens (vs 1,024 for BART)
- **Performance**: Best for scientific papers, legal documents, books
- **Model Card**: [allenai/led-large-16384](https://huggingface.co/allenai/led-large-16384)
- **Paper**: [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150)
- **Size**: 1.6GB (406M parameters)

#### 5. 📖 LongT5
**For very long document summarization**
- **Why it's special**: Extends T5 to handle 16K+ tokens efficiently
- **Performance**: Excellent on long-form content like research papers
- **Model Card**: [google/long-t5-tglobal-large](https://huggingface.co/google/long-t5-tglobal-large)
- **Paper**: [LongT5: Efficient Text-To-Text Transformer for Long Sequences](https://arxiv.org/abs/2112.07916)
- **Size**: 3GB (770M parameters)

### Why Not Covered?

These models require:
- **GPU Memory**: 12-24GB VRAM (RTX 3090/4090 or better)
- **Inference Time**: 10-60 seconds per summary (vs 2-5 seconds for DistilBART)
- **Disk Space**: 1.6-11GB per model
- **RAM**: 16-32GB system memory

For learning purposes, DistilBART provides 90% of the quality at 10% of the resource cost!

### Learning Path Recommendation

1. **Start here**: Master DistilBART (this notebook)
2. **Next step**: Try BART-large if you have 8GB+ VRAM
3. **Advanced**: Experiment with PEGASUS for news summarization
4. **Specialized**: Use LED/LongT5 for long documents only when needed

### Benchmarks & Leaderboards

- **ROUGE Scores** (CNN/DailyMail dataset):
  - DistilBART: ROUGE-2 ~19.5
  - BART-large: ROUGE-2 ~21.3
  - PEGASUS: ROUGE-2 ~21.9
  - T5-3B: ROUGE-2 ~22.5

- **Explore benchmarks**: [Papers With Code - Text Summarization](https://paperswithcode.com/task/text-summarization)

### Quick Comparison Table

| Model | Size | Speed | Quality | Best For |
|-------|------|-------|---------|----------|
| **DistilBART** ⭐ | 1.2GB | Fast | Good | Learning, CPU inference |
| **BART-large** | 1.6GB | Medium | Great | General summarization |
| **PEGASUS** | 2.3GB | Slow | Excellent | News articles |
| **T5-Large** | 3GB | Slow | Excellent | Diverse tasks |
| **LED** | 1.6GB | Very Slow | Great | Long documents (10K+ words) |
| **LongT5** | 3GB | Very Slow | Excellent | Very long documents |

**💡 Tip**: For production applications with quality requirements, PEGASUS or T5 are worth the computational investment!

## Exercises

1. **Length Experiment**: Try different `max_length` values. When does the summary become too short or too long?

2. **Beam Search**: Experiment with `num_beams` parameter. Compare quality with 1, 4, and 8 beams.

3. **Custom Articles**: Find a news article online and summarize it. How well does the model do?

4. **Model Comparison**: If you have GPU, compare `distilbart` with `bart-large-cnn`. Notice quality differences?

5. **Long Documents**: Test with very long documents (500+ words). Does the model handle them well?

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **Abstractive summarization** generates new sentences rather than extracting existing ones

✅ **Length parameters** control summary size (`max_length`, `min_length`)

✅ **Beam search** (`num_beams`) improves quality at the cost of speed

✅ **Batch processing** is efficient for multiple documents

✅ BART models are specifically trained for summarization tasks

## Next Steps

- Try **Notebook 04**: Image Classification for computer vision
- Explore other summarization models on [HuggingFace Hub](https://huggingface.co/models?pipeline_tag=summarization)
- Learn about extractive summarization approaches

## Resources

- [Summarization Task Guide](https://huggingface.co/docs/transformers/tasks/summarization)
- [BART Paper](https://arxiv.org/abs/1910.13461)
- [Summarization Metrics](https://huggingface.co/spaces/evaluate-metric/rouge)